# Kindle Book Recommender System

### Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import warnings
warnings.filterwarnings('ignore')

In [2]:
pd.reset_option('display.max_rows')

In [3]:
def getthirdValue(aList):
    return aList[2:3]

In [4]:
def meta_clean(data):
    data.drop(columns=['tech1', 'description', 'fit', 'tech2',
                    'feature', 'rank', 'also_view', 'main_cat',
                    'similar_item', 'date', 'price', 'imageURL',
                    'imageURLHighRes', 'also_buy'], inplace=True)
    data['category'] = data['category'].apply(eval)
    data['genre'] = data['category'].apply(getthirdValue)
    data['genre'] = data['genre'].apply(lambda x: ','.join(map(str, x)))
    data['genre'] = data['genre'].str.replace('amp;', '')
    data.drop(columns=['category'], inplace=True)
    data['details'] = data['details'].apply(eval)
    print_length = [d.get('Print Length:') for d in data['details']]
    data['print_length'] = print_length
    data['print_length'] = data['print_length'].str.extract('(\d+)', expand=False)
    data['print_length']= data['print_length'].astype(float)
    word_wise = [d.get('Word Wise:') for d in data['details']]
    data['word_wise'] = word_wise
    lending = [d.get('Lending:') for d in data['details']]
    data['lending'] = lending
    bool_dict = {'Enabled': 1, 'Not Enabled': 0}
    data['word_wise'] = data['word_wise'].map(bool_dict)
    data['lending'] = data['lending'].map(bool_dict)
    data['brand'] = data['brand'].str.replace("Visit Amazon's", '')
    data['brand'] = data['brand'].str.replace("Page", '')
    data.drop(columns=['details'], inplace=True)
    data['title'].str.replace("&#39;", "'")
    data.rename(columns={'brand': 'author'}, inplace=True)
    return data.head()

In [5]:
path = 'C:\\Users\\danie\\Documents\\Flatiron\\Projects\\Capstone\\Rawdata\\meta_Kindle_Store.gz'
df_meta = pd.read_json(path, compression='gzip', lines=True)

In [6]:
path = 'C:\\Users\\danie\\Documents\\Flatiron\\Projects\\Capstone\\Rawdata\\Kindle_Store_5.gz'
df_rev = pd.read_json(path, compression='gzip', lines=True)

In [7]:
df_meta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 491670 entries, 0 to 491669
Data columns (total 19 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   category         491670 non-null  object        
 1   tech1            491670 non-null  object        
 2   description      491670 non-null  object        
 3   fit              491670 non-null  object        
 4   title            491670 non-null  object        
 5   also_buy         491670 non-null  object        
 6   tech2            491670 non-null  object        
 7   brand            491670 non-null  object        
 8   feature          491670 non-null  object        
 9   rank             491670 non-null  object        
 10  also_view        491670 non-null  object        
 11  details          491670 non-null  object        
 12  main_cat         491670 non-null  object        
 13  similar_item     491670 non-null  object        
 14  date             0 n

In [8]:
df_meta['category'].value_counts()

TypeError: unhashable type: 'list'

Exception ignored in: 'pandas._libs.index.IndexEngine._call_map_locations'
Traceback (most recent call last):
  File "pandas\_libs\hashtable_class_helper.pxi", line 5231, in pandas._libs.hashtable.PyObjectHashTable.map_locations
TypeError: unhashable type: 'list'


[Kindle Store, Kindle eBooks, Literature & Fiction]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

In [9]:
df_meta['category'] = df_meta['category'].astype('str')

In [10]:
df_meta = df_meta.loc[
    lambda df: df.category.apply(
        lambda l: 'Kindle eBooks' in l)]

In [11]:
df_meta['details'] = df_meta['details'].astype('str')

In [12]:
details = list(df_meta['details'])

In [13]:
details[0]

"{'File Size:': '295 KB', 'Print Length:': '113 pages', 'Publisher:': 'FuturesPast Editions (October 14, 2001)', 'Publication Date:': 'October 14, 2001', 'Language:': 'English', 'ASIN:': 'B000FA5KKA', 'Word Wise:': 'Enabled', 'Lending:': 'Enabled'}"

In [14]:
meta_clean(df_meta)

,title,author,asin,genre,print_length,word_wise,lending
0,,Arthur K. Barnes,B000FA5KKA,Science Fiction & Fantasy,113.0,1.0,1.0
1,,Paul A. Craig,B000FA5M3K,Engineering & Transportation,268.0,0.0,0.0
2,,Jean Marie Stine,B000FA5KJQ,Biographies & Memoirs,103.0,1.0,1.0
3,,Arthur K. Barnes,B000FA5NSO,Science Fiction & Fantasy,116.0,0.0,1.0
4,,Ethan M. Rasiel,B000FA5KX2,Business & Money,187.0,1.0,0.0


In [15]:
df_meta.genre.value_counts()

Literature & Fiction                           149570
Religion & Spirituality                         48975
Romance                                         34401
Science Fiction & Fantasy                       29383
Children's eBooks                               27427
Business & Money                                25014
Health, Fitness & Dieting                       19009
Mystery, Thriller & Suspense                    14910
History                                         13544
Biographies & Memoirs                           11759
Cookbooks, Food & Wine                          10796
Politics & Social Sciences                       8885
Teen & Young Adult                               8754
Education & Teaching                             8711
Humor & Entertainment                            7898
Arts & Photography                               7652
Sports & Outdoors                                7356
Crafts, Hobbies & Home                           6041
Comics, Manga & Graphic Nove

In [16]:
df_meta = df_meta[df_meta['genre'].map(df_meta['genre'].value_counts()) > 1000]
df_meta = df_meta.loc[df_meta['genre'] != '']

In [17]:
df_meta = df_meta.replace(r'^\s*$', np.nan, regex=True)

In [18]:
df_meta.isna().sum()

title           2566
author           899
asin               0
genre              0
print_length    8247
word_wise        348
lending          348
dtype: int64

In [19]:
df_meta['print_length'] = df_meta.groupby(['genre'], sort=False)['print_length'].apply(lambda x: x.fillna(x.mean()))

In [20]:
df_meta.dropna(inplace=True)

In [21]:
df_meta.asin.value_counts()

B00128EFK0    1
B00QMTZSJ6    1
B00QMUNOC8    1
B00QMUNO32    1
B00QMUNN7O    1
             ..
B00CU4YVAE    1
B00CU581C2    1
B00CU4RSOA    1
B00CU4Z3H4    1
B01HJH82WY    1
Name: asin, Length: 482582, dtype: int64

In [22]:
asin_list = df_meta['asin'].tolist()

In [23]:
df_rev.head()

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image
0,4,True,"07 3, 2014",A2LSKD2H9U8N0J,B000FA5KK0,{'Format:': ' Kindle Edition'},sandra sue marsolek,"pretty good story, a little exaggerated, but I...",pretty good story,1404345600,NaN,NaN
1,5,True,"05 26, 2014",A2QP13XTJND1QS,B000FA5KK0,{'Format:': ' Kindle Edition'},Tpl,"If you've read other max brand westerns, you k...",A very good book,1401062400,NaN,NaN
2,5,True,"09 16, 2016",A8WQ7MAG3HFOZ,B000FA5KK0,{'Format:': ' Kindle Edition'},Alverne F. Anderson,"Love Max, always a fun twist",Five Stars,1473984000,NaN,NaN
3,5,True,"03 3, 2016",A1E0MODSRYP7O,B000FA5KK0,{'Format:': ' Kindle Edition'},Jeff,"As usual for him, a good book",a good,1456963200,NaN,NaN
4,5,True,"09 10, 2015",AYUTCGVSM1H7T,B000FA5KK0,{'Format:': ' Kindle Edition'},DEHS - EddyRapcon,MB is one of the original western writers and ...,A Western,1441843200,2,NaN


In [24]:
df_rev = df_rev[df_rev['asin'].isin(asin_list)]

In [25]:
df_rev.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2198420 entries, 2741 to 2222982
Data columns (total 12 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   overall         int64 
 1   verified        bool  
 2   reviewTime      object
 3   reviewerID      object
 4   asin            object
 5   style           object
 6   reviewerName    object
 7   reviewText      object
 8   summary         object
 9   unixReviewTime  int64 
 10  vote            object
 11  image           object
dtypes: bool(1), int64(2), object(9)
memory usage: 203.4+ MB


In [26]:
df_rev.verified.value_counts()

True     1399604
False     798816
Name: verified, dtype: int64

In [27]:
indexNames = df_rev[df_rev['verified'] == False].index
df_rev.drop(indexNames , inplace=True)

In [28]:
df_rev_use = df_rev.drop(columns=['reviewTime', 'verified',
                                  'style', 'reviewerName', 
                                  'unixReviewTime', 'image', 'vote', 'summary'])

In [29]:
df_rev_use.isna().sum()

overall         0
reviewerID      0
asin            0
reviewText    319
dtype: int64

In [30]:
df_rev_use.dropna(inplace=True)

In [31]:
df_rev_use['reviewerID'].value_counts()

A2UNMDJYXPEQZ3    717
A3IQ0P3M39IY8U    716
A37LY77Q2YPJVL    653
A26GE9ZG68SKIQ    618
A2VXSQHJWZAQGY    614
                 ... 
AFRG7LGFKNYOV       1
A16KAI92YFBP4X      1
A39QLU7TB2PL4C      1
A2HJDKYR5TFEGR      1
A3F3TCUETH67WY      1
Name: reviewerID, Length: 131057, dtype: int64

In [32]:
df_rev_use['asin'].value_counts()

B00YN6XHMU    1947
B00ESJ3S94    1110
B00C2WDD5I     901
B00IJYII4E     795
B015BIHKH6     724
              ... 
B00H27S8V2       1
B00H2LZOOC       1
B017HRAYYK       1
B00H2NDGJU       1
B01HJENY3Y       1
Name: asin, Length: 94211, dtype: int64

In [33]:
df_rev_use.to_csv('Data/df_rev_all.csv')

In [34]:
df_rev5 = df_rev_use[df_rev_use['reviewerID'].map(df_rev_use['reviewerID'].value_counts()) > 4]

In [35]:
df_rev5['reviewerID'].value_counts()

A2UNMDJYXPEQZ3    717
A3IQ0P3M39IY8U    716
A37LY77Q2YPJVL    653
A26GE9ZG68SKIQ    618
A2VXSQHJWZAQGY    614
                 ... 
AZEQFNT786AMH       5
A8W4ZYLIS1ZAX       5
A16PDUVGEHUSQS      5
A5ETYMZJL0TAU       5
A3I36S1KLJE1H6      5
Name: reviewerID, Length: 98042, dtype: int64

In [36]:
df_rev5.to_csv('Data/df_rev5.csv')

In [37]:
asin_set = set(df_rev['asin'].tolist())

In [38]:
asin_set5 = set(df_rev5['asin'].tolist())

In [39]:
len(asin_set)

94212

In [40]:
len(asin_set5)

93734

In [41]:
df_meta_all = df_meta.loc[df_meta['asin'].isin(asin_set)]

In [42]:
df_meta5 = df_meta.loc[df_meta['asin'].isin(asin_set5)]

In [43]:
df_meta_all.to_csv('Data/meta_all.csv')
df_meta5.to_csv('Data/meta5.csv')

In [44]:
popular = df_rev_use.groupby(by = 'asin').count()
popular = popular.sort_values(by = 'overall', ascending=False)

In [45]:
popular.head()

,overall,reviewerID,reviewText
asin,,,
B00YN6XHMU,1947,1947,1947
B00ESJ3S94,1110,1110,1110
B00C2WDD5I,901,901,901
B00IJYII4E,795,795,795
B015BIHKH6,724,724,724


In [46]:
df_most = df_rev5.loc[df_rev5['reviewerID'] == 'A2UNMDJYXPEQZ3']
df_most['overall'].value_counts()

5    266
4    193
3    157
2     76
1     25
Name: overall, dtype: int64

In [47]:
df_2most = df_rev5.loc[df_rev5['reviewerID'] == 'A3IQ0P3M39IY8U']
df_2most['overall'].value_counts()

5    292
4    234
3    140
2     42
1      8
Name: overall, dtype: int64

In [48]:
df_3most = df_rev5.loc[df_rev5['reviewerID'] == 'A37LY77Q2YPJVL']
df_3most['overall'].value_counts()

5    518
4    135
Name: overall, dtype: int64

In [49]:
df_4most = df_rev5.loc[df_rev5['reviewerID'] == 'A26GE9ZG68SKIQ']
df_4most['overall'].value_counts()

4    535
5     57
3     25
2      1
Name: overall, dtype: int64

In [50]:
df_5most = df_rev5.loc[df_rev5['reviewerID'] == 'A2VXSQHJWZAQGY']
df_5most['overall'].value_counts()

3    248
4    237
2    104
5     15
1     10
Name: overall, dtype: int64

In [51]:
df_books_rev = df_rev_use.groupby(['asin'], as_index = False).agg({'reviewText': ' '.join})
df_books_rev.to_csv('Data/df_books_rev.csv')

In [52]:
df_books_rev.head()

,asin,reviewText
0,B0012GTZCK,Not what I was expecting. I was looking for a ...
1,B0012LHGJ4,IT WAS DONE AS A FILM IN THE 1950-60s. VERY P...
2,B0012RMVH0,Needs pictures Excellent book. This is somethi...
3,B0012TAD1O,She always delivers with an exciting storyline...
4,B0012U0NKE,"<a data-hook=""product-link-linked"" class=""a-li..."


In [53]:
df_books_rev.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94211 entries, 0 to 94210
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   asin        94211 non-null  object
 1   reviewText  94211 non-null  object
dtypes: object(2)
memory usage: 1.4+ MB
